#### Agents

1. Core idea to use language model to choose sequence of actions to take.

2. In chains, the sequence of actions were hardcoded.

3. In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.



In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

SECRET_KEY = os.environ.get('OPEN_AI_KEY')

In [ ]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(openai_api_key = SECRET_KEY)

In [ ]:
# Agent Type - OpenAI Function

#%pip install duckduckgo-search 
#% pip install langchainhub

# create agent
from langchain import hub
prompt = hub.pull('hwchase17/openai-functions-agent')

# print(prompt)

from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import create_openai_functions_agent, AgentExecutor

search = DuckDuckGoSearchRun()
tools = [search]
agent = create_openai_functions_agent(
    llm = chat,
    tools = tools, 
    prompt = prompt
)

agent_executor = AgentExecutor(agent = agent,
                               tools = tools,
                               verbose = True)

response = agent_executor.invoke(
    {'input' : 'Who is the current Prime Minister of India ? '}
)
print(response)

In [ ]:
# using duck duck go independenly
from langchain_community.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

results = search.run("Give me world finance news ")
results

#### ReAct Agent

1. what is ReAct Agent : `https://react-lm.github.io`

In [ ]:
# React Agent

from langchain.agents import create_react_agent,AgentExecutor

prompt = hub.pull("hwchase17/react")
print("----Prompt Looks like this " ,prompt)

search = DuckDuckGoSearchRun()

tools = [search]
agent = create_react_agent(llm = chat,
                           tools = tools,
                           prompt = prompt)

# Run agent
agent_executor = AgentExecutor(agent = agent,
                               tools = tools,
                               verbose = True)

response = agent_executor.invoke(
    {"input" : "Who is the current Prime Minister of India ? "}
)

print(response)

#### Tools

1. There are prebuild tools as well as,

2. You can create your custom tools too

In [20]:
# Custom Tool using decorator
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def search(query:str)->str:
    """Look up  things online"""
    return search.run(query)

print(search.name)
print(search.args)
print(search.description)

# can use it like this
tools = [search] # no need of creating an object


search
{'query': {'title': 'Query', 'type': 'string'}}
search(query: str) -> str - Look up  things online


In [23]:
# To make custom tool more better, an inherit the BaseModel
class SearchInput(BaseModel):
    query:str = Field(description = "should be a search query")
@tool('search-tool', args_schema=SearchInput,return_direct=True)
def search(query:str)->str:
    """Look up  things online"""
    return search.run(query)

print(search.name)
print(search.args)
print(search.description)

search-tool
{'query': {'title': 'Query', 'description': 'should be a search query', 'type': 'string'}}
search-tool(query: str) -> str - Look up  things online


In [25]:
# using StructuredTool

class CalculatorInput(BaseModel):
    a:int = Field(description="first number ")
    b:int = Field(description="second number ")
    
def multiply(a:int, b:int) ->int:
    """Multiply two numbers"""
    return a * b

cal = StructuredTool.from_function(
    func = multiply,
    name = "Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True
)

print(cal.name)
print(cal.args)
print(cal.description)

Calculator
{'a': {'title': 'A', 'description': 'first number ', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second number ', 'type': 'integer'}}
Calculator(a: int, b: int) -> int - multiply numbers


In [ ]:
# using custom tool 

# 1.Defining custom tool
@tool
def get_word_length(words:str)->int:
    """Returns the length of a string"""
    return len(words)

# 2. Create Agent
prompt = hub.pull('hwchase17/react')

# 3.
tools = [get_word_length]

#4. 
agent = create_react_agent(
    llm = chat,
    tools = tools,
    prompt = prompt
)

agent_executor = AgentExecutor(
    agent = agent,
    tools=tools,
    verbose= True
)

#5. 
response = agent_executor.invoke(
    {"input" : "Hello"}
)

print(response)

# through agent like these you can do api call, database entry..


In [ ]:
# Using multiple tools

@tool
def get_word_length(words:str)->int:
    """Returns the length of a string"""
    return len(words)

prompt = hub.pull('hwchase17/react')
search = DuckDuckGoSearchRun()

# 3.
tools = [get_word_length,search]

#4. 
agent = create_react_agent(
    llm = chat,
    tools = tools,
    prompt = prompt
)

agent_executor = AgentExecutor(
    agent = agent,
    tools=tools,
    verbose= True
)

#5. 
response = agent_executor.invoke(
    {"input" : "What is LangChain ? What is the total word length of that including spaces ? "}
)

#### Using Wikipedia tool

In [26]:
%pip install wikipedia

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=9ccd152a640f648b0c5ba4debda1fc871372c4d9543e0b9488d7b1700121f432
  Stored in directory: /Users/singhabhishekkk/Library/Caches/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper = WikipediaAPIWrapper())
wikipedia.run("Cancer")

"Page: Cancer\nSummary: Cancer is a group of diseases involving abnormal cell growth with the potential to invade or spread to other parts of the body. These contrast with benign tumors, which do not spread. Possible signs and symptoms include a lump, abnormal bleeding, prolonged cough, unexplained weight loss, and a change in bowel movements. While these symptoms may indicate cancer, they can also have other causes. Over 100 types of cancers affect humans.Tobacco use is the cause of about 22% of cancer deaths. Another 10% are due to obesity, poor diet, lack of physical activity or excessive alcohol consumption. Other factors include certain infections, exposure to ionizing radiation, and environmental pollutants. Infection with specific viruses, bacteria and parasites is an environmental factor causing approximately 16-18% of cancers worldwide.  These infectious agents include Helicobacter pylori, hepatitis B, hepatitis C, human papillomavirus infection, Epstein–Barr virus, Human T-ly